Normalización del dataset combinando los aspectos mencionados en las especificaciones

In [1]:
# Añadir el directorio 'src' al sys.path
import sys
import os

In [2]:
# Obtener la ruta absoluta de la carpeta 'src' para que sea accesible desde el notebook
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

In [3]:
# Importar la función de normalización
from preprocessing import normalize_text

In [4]:
# Importar pandas para cargar y manipular el dataset
import pandas as pd

In [5]:
# Cargar el dataset
df = pd.read_csv("../data/TA1C_dataset_detection_train.csv")


In [6]:
# Aplicar cada tipo de normalización
df['tokenized_text'] = df['Teaser Text'].apply(lambda x: normalize_text(x, mode="tokenization"))
df['cleaned_text'] = df['Teaser Text'].apply(lambda x: normalize_text(x, mode="text_cleaning"))
df['no_stopwords_text'] = df['Teaser Text'].apply(lambda x: normalize_text(x, mode="remove_stopwords"))
df['lemmatized_text'] = df['Teaser Text'].apply(lambda x: normalize_text(x, mode="lemmatization"))


In [7]:
# Mostrar ejemplos de cada tipo de normalización
print("\nEjemplos de normalización:")
print(df[['Teaser Text', 'tokenized_text', 'cleaned_text', 'no_stopwords_text', 'lemmatized_text']].head())



Ejemplos de normalización:
                                         Teaser Text  \
0  #SegundaDivisión  | La fortaleza del ataque: R...   
1  Jorge Lanata a los argentinos que se van a Uru...   
2  Raffo: “Los montevideanos deben estar alerta p...   
3  Ecos del universo: joven uruguayo desentraña (...   
4  Propuesta quinquenal de ANEP: aumento de 3,8% ...   

                                      tokenized_text  \
0  [#, SegundaDivisión,  , |, La, fortaleza, del,...   
1  [Jorge, Lanata, a, los, argentinos, que, se, v...   
2  [Raffo, :, “, Los, montevideanos, deben, estar...   
3  [Ecos, del, universo, :, joven, uruguayo, dese...   
4  [Propuesta, quinquenal, de, ANEP, :, aumento, ...   

                                        cleaned_text  \
0  segundadivisión   la fortaleza del ataque ramp...   
1  jorge lanata a los argentinos que se van a uru...   
2  raffo “los montevideanos deben estar alerta po...   
3  ecos del universo joven uruguayo desentraña y ...   
4  propuesta quinq

In [8]:

# Guardar el dataset con todas las columnas de normalización
output_path = "../data/TA1C_dataset_detection_train_cleaned.csv"
df.to_csv(output_path, index=False)
print(f"Dataset con normalizaciones guardado en '{output_path}'")

Dataset con normalizaciones guardado en '../data/TA1C_dataset_detection_train_cleaned.csv'


Dividimos el dataset dentro del Notebook

In [9]:
# Guardar el dataset con todas las columnas de normalización
output_path = "../data/TA1C_dataset_detection_train_cleaned.csv"
df.to_csv(output_path, index=False)
print(f"Dataset con normalizaciones guardado en '{output_path}'")

Dataset con normalizaciones guardado en '../data/TA1C_dataset_detection_train_cleaned.csv'


In [10]:
from sklearn.model_selection import train_test_split

# Dividir el dataset en conjuntos de entrenamiento y desarrollo
train_df, dev_df = train_test_split(
    df,
    test_size=0.25,  # 25% para desarrollo
    random_state=0,  # Semilla fija para reproducibilidad
    stratify=df['Tag Value']  # Estratificar según la columna de clase
)

# Guardar los conjuntos en archivos CSV
train_df.to_csv("../data/TA1C_dataset_detection_train_split.csv", index=False)
dev_df.to_csv("../data/TA1C_dataset_detection_dev_split.csv", index=False)

print("Conjuntos de entrenamiento y desarrollo guardados en '../data/'")

Conjuntos de entrenamiento y desarrollo guardados en '../data/'


Representación de texto

In [11]:
# Cargar los conjuntos de entrenamiento y desarrollo
train_df = pd.read_csv("../data/TA1C_dataset_detection_train_split.csv")
dev_df = pd.read_csv("../data/TA1C_dataset_detection_dev_split.csv")

# Verificar las columnas disponibles
print(train_df.columns)

Index(['Tweet ID', 'Tweet Date', 'Media Name', 'Media Origin', 'Teaser Text',
       'Tag Value', 'tokenized_text', 'cleaned_text', 'no_stopwords_text',
       'lemmatized_text'],
      dtype='object')
